# CLEANING

## CSV - DATASET

In [1]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.simplefilter('ignore')
import datetime

In [2]:
df=pd.read_csv('../data/results.csv')

In [3]:
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False
4,1975-08-25,Thailand,Australia,3,2,AFC Championship,Hong Kong,Hong Kong,True


In [4]:
df.shape

(4314, 9)

In [5]:
def dataframe_cleaning(df):
    ''' Changing date to year in order to work with years '''
    df2['date'] = df['date'].astype('datetime64')
    df2['date'] = df['date'].dt.year
    df2.rename(columns = {'date':'year'}, inplace = True)
    return df2

In [6]:
##def drop(df):
   ## ''' As [tournament],[neutral],[city] and [country] columns are not interesting for our purpose, we'll drop them '''
   ## df.drop(columns =['tournament','neutral','city','country'], inplace=True)
   ## return df##
#    df2 = df.drop(columns =['neutral'], inplace=True) 

In [7]:
def drop22(df):
    ''' Changing date to year in order to work with years '''
    df.drop(['neutral'],axis=1,inplace=True)
    return df

In [8]:
def drops(df):
    ''' Changing date to year in order to work with years '''
    df = df.drop(columns =['neutral'], inplace=True) 
    return df 

In [9]:
df.dtypes

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object

In [10]:
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False
4,1975-08-25,Thailand,Australia,3,2,AFC Championship,Hong Kong,Hong Kong,True


In [11]:
df = df.drop('neutral',axis=1)

In [12]:
df =df.loc[df["year"].between(2000, 2021)]
df.reset_index(drop=True, inplace=True)
df.head(5)

KeyError: 'year'

In [ ]:
df['result']=df.home_score - df.away_score

In [ ]:
df.result=df.result.apply(ganar)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df['winner'] = pd.Series(dtype='str')
df.winner = df.apply(lambda x : x['home_team'] if x['home_score'] > x['away_score'] 
                                              else (x['away_team'] if x['home_score'] < x['away_score'] else 'draw'),
                                              axis=1)
df.head()

In [ ]:
df2 = df[['year', 'winner']].copy()

In [ ]:
df2.to_csv('./data/df2.csv', index=False)

In [ ]:
## not used .... df['points'] = pd.Series(dtype='int')

# SCRAPING

## Method: Selenium

In [ ]:
%pip install selenium
from selenium import webdriver

In [ ]:
%pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
from selenium.webdriver.chrome.options import Options

opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
opciones.add_extension('driver/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [ ]:
PATH=ChromeDriverManager().install()
URL='https://www.goal.com/en-in/news/fifa-womens-world-player-of-the-year-award-winners/blt12683887fe8df959'
driver=webdriver.Chrome(PATH)
driver.get(URL)

In [ ]:
acepto=driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')

acepto.click()

In [ ]:
tabla = driver.find_element_by_xpath('//*[@id="__next"]/div/main/div/div/article/main/div[3]/div/div[4]')

players = tabla.find_elements_by_tag_name('tr')

players

In [ ]:
lst = []

for i in players:
    i = i.text.replace('\n', ',,,').replace('  ', ',,,').split(',,,')
    lst.append(i)
lst

In [ ]:
import pandas as pd

col_names=lst[0]

data=lst[1:]

df3=pd.DataFrame(data, columns=col_names)

df3.head()

In [ ]:
df3['Club']=[c.replace('Carolina Courage/Birgit Prinz', 'FFC Frankfurt') for c in df3.Club]

df3

## BEAUTIFUL SOUP

In [ ]:
import requests as req

from bs4 import BeautifulSoup as bs
url='https://www.topendsports.com/sport/soccer/list-player-of-the-year-women.htm'
html=req.get(url).content

html[:1000]

In [ ]:
soup=bs(html, 'html.parser')

tabla=soup.find('div',{'class':'table-responsive'})
tabla
type(tabla)

In [ ]:
tabla2=tabla.find('table',{'class':'list'})
tabla2

In [ ]:
filas=tabla2.find_all('tr')

filas=[f.text.strip().split('\n') for f in filas]

In [ ]:
col_names=filas[0]

data=filas[1:]

df4=pd.DataFrame(data, columns=col_names)

df4.head()

In [ ]:
df4 = df4.rename(str.title, axis='columns')
df4.head(1)

In [ ]:
df4 = df4.sort_values(by = ['Year'], ascending=True).reset_index().drop('index',axis=1)
df4.head()

In [ ]:
df4['Player']=[c.replace('Nadine Keßler', 'Nadine Kebler') for c in df4.Player]

df4['Player'] = df4['Player'].apply(lambda x: x.strip())
df4

In [ ]:
df5 = pd.merge(df3,df4,how="inner")
df5

In [ ]:
df5['Nationality']=[c.replace('USA', 'United States') for c in df5.Nationality]
df5['Nationality']=[c.replace('The Netherlands', 'Netherlands') for c in df5.Nationality]

In [ ]:
df5['Club'].value_counts()

In [ ]:
df5

In [ ]:
df5.to_csv('./data/players.csv', index=False)

# Selenium

In [ ]:
PATH=ChromeDriverManager().install()
URL='https://www.worldfootball.net/winner/frauen-champions-league/'
driver=webdriver.Chrome(PATH)
driver.get(URL)

acepto=driver.find_element_by_xpath('//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]')
acepto.click()

In [ ]:
tabla = driver.find_element_by_xpath('//*[@id="site"]/div[2]/div[1]/div[1]')
tabla

In [ ]:
players = tabla.find_elements_by_tag_name('tr')
players

In [ ]:
lst = []

for i in players:
    i = i.text.replace('\n', ',').replace('   ', ',,,').split(' ')
    lst.append(i)
lst

lst[0].append('a')
lst[0].append('b')
lst

In [ ]:
col_names=lst[0]

data=lst[1:]

cham=pd.DataFrame(data, columns=col_names)

cham

In [ ]:
cham["Champion"] = cham["Winner"] + " " + cham["Country"]
cham

In [ ]:
cham['Champion']=[c.replace('1. FFC', 'FFC Frankfurt') for c in cham.Champion]
cham['Champion']=[c.replace('1.', 'FFC Frankfurt') for c in cham.Champion]
cham = cham.drop(['Winner','Country'], axis=1)
cham.head()

In [ ]:
cham['a']=[c.replace('Frankfurt', 'Germany') for c in cham.a]
cham['a']=[c.replace('Duisburg', 'Germany') for c in cham.a]
cham['a']=[c.replace('FF', 'Sweden') for c in cham.a]
cham = cham.drop('b', axis=1)
cham.head()

In [ ]:
cham.rename(columns = {'a':'Country'}, inplace = True)
cham = cham.sort_values(by = ['Year'], ascending=True).reset_index().drop('index',axis=1)
cham['Champion']=[c.replace('Umeå IK', 'Umea IK') for c in cham.Champion]
cham['Champion']=[c.replace('VfL Wolfsburg', 'Wolfsburg') for c in cham.Champion]
cham

In [ ]:
cham.to_csv('./data/champions.csv', index=False)